<h1 align="center"><b>player Activity</b></h1>

<h5>
    <b> Packages </b>
</h5> 

In [1]:
# Package used to connect to MySQL Databases
import mysql.connector

# XML Creation
import xml.etree.ElementTree as ET
import os

# Data Manipulation Packages
import pandas as pd
import numpy as np
import hashlib
from datetime import datetime

# Package To Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

<h5>
    <b> Import Data From MySQL</b>
</h5> 

In [9]:
# Code To Connect MySQL
file = open('/Users/katlegomatebane/Documents/Connect/Connect.txt', 'r')
text = file.readlines()

cobi_betika = mysql.connector.connect(host=text[0].strip()
                                      ,database=text[7].strip()
                                      ,user=text[5].strip()
                                      ,password=text[6].strip()
                                      ,port=text[4].strip())

# Connect to MySQL database
try:
    with cobi_betika.cursor() as cursor:
        df = pd.read_sql("SELECT x.profile_id \
                                ,(x.opening_depostit_amt - x.opening_withdrawal_amt) AS openingBalance\
                                ,(x.closing_depostit_amt - x.closing_withdrawal_amt) AS closingBalance\
                                ,x.depostit_amt\
                                ,x.withdrawal_amt\
                                ,x.summary_date\
                                ,x.registration_date\
                                ,x.payment_method\
\
                            FROM (SELECT a.profile_id \
                                        ,SUM(CASE WHEN DATE(a.summary_date) <= DATE(CURDATE()- INTERVAL 2 DAY) THEN a.deposit_amt END)      AS opening_depostit_amt\
                                        ,SUM(CASE WHEN DATE(a.summary_date) <= DATE(CURDATE()- INTERVAL 2 DAY) THEN a.withdrawal_amt END)   AS opening_withdrawal_amt\
                                        ,SUM(CASE WHEN DATE(a.summary_date) <= DATE(CURDATE()- INTERVAL 1 DAY) THEN a.deposit_amt END)      AS closing_depostit_amt\
                                        ,SUM(CASE WHEN DATE(a.summary_date) <= DATE(CURDATE()- INTERVAL 1 DAY) THEN a.withdrawal_amt END)   AS closing_withdrawal_amt\
                                        ,SUM(CASE WHEN DATE(a.summary_date) =  DATE(CURDATE()- INTERVAL 1 DAY) THEN a.deposit_amt END)      AS depostit_amt\
                                        ,SUM(CASE WHEN DATE(a.summary_date) =  DATE(CURDATE()- INTERVAL 1 DAY) THEN a.withdrawal_amt END)   AS withdrawal_amt\
                                        ,MAX(CASE WHEN DATE(a.summary_date) =  DATE(CURDATE()- INTERVAL 1 DAY) THEN a.summary_date END)     AS summary_date\
                                        ,b.registration_date\
                                        ,a.payment_method\
                                FROM betika_bi_gh.f_cashier_gh AS a \
\
                                LEFT JOIN betika_bi_gh.dim_first_last_gh AS b\
                                ON a.profile_id = b.profile_id\
\
                                WHERE a.profile_id IN (1341048)\
\
                                GROUP BY profile_id\
                                        ,b.registration_date\
                                        ,a.payment_method\
                                ) AS x; "
                         ,cobi_betika)
        
finally:
    cobi_betika.close()

<h5>
    <b> Create File Variables </b>
</h5> 

In [3]:
report_path = '/Users/katlegomatebane/Documents/UG - EMS Integration/Reports/player Activity/'
name        = '?xml version="1.0" encoding="UTF-8" standalone="yes"?'
date        = df['summary_date'].min().strftime('%Y%m%d')
counter     = 1
title       = 'RRI_playerActivity' 

<h5>
    <b> Create Serialisation </b>
</h5> 

In [4]:
for file in os.listdir(report_path):
        if "xml" in file:
            counter += 1

serial = f"{counter:03}"

In [10]:
df

,profile_id,openingBalance,closingBalance,depostit_amt,withdrawal_amt,summary_date,registration_date,payment_method
0,1341048,777.52,732.52,150.0,195.0,2024-06-16,2021-01-24,MTN


<h5>
    <b> Create Data Variables </b>
</h5> 

In [33]:
df['opener']            = '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>'
df['xmlns:rri']         = 'urn:GSA:RRI:1.0:GSA:rri'
df['xmlns:xsi']         = 'http://www.w3.org/2001/XMLSchema-instance'
df['version']           = '1.0'
df['partionid']         = '1'
df['SeqNumber']         = '1'
df['OperatorId']        = 'Betika'
df['periodType']        = 'RRI_day'
df['reportType']        = 'RRI_playerActivity'
df['dateTime']          = datetime.now().strftime('%Y-%m-%dT%M:%S.%T')


# PlayerActivity
df['brandID']           = 'Betika'

# PlayerActivityDetail
df['ActivityDescOW']    = 'Player Account Opened'
df['ActivityDescDP']    = 'RRI_deposit'
df['ActivityDescWD']    = 'RRI_withdrawal'
df['fundsType']         = 'RRI_realMoney'
df['currencyCode']      = 'UGX'
df['transClass']        = 'RRI_online'

<h5>
    <b> Prepare Data To Be Hashed By Game ID </b>
</h5> 

In [34]:
# Create New dataframe that will be used to group data by game id
df_Hash = df

# Place all the row's data into one column
df_Hash['data'] = str(df_Hash).encode()

# convert data column to string 
df_Hash['data'] = df_Hash['data'].astype(str)

# 
df_Hash['data'] = df_Hash.groupby(['summary_date'])['data'].transform(lambda x: ', '.join(x))

# 
df_Hash = df_Hash.groupby('summary_date').first().reset_index()

# Keep only the data field & game id
df_Hash = df_Hash[['summary_date','data']]

In [35]:
def hash_data(data):
    
    # Encode the transaction data as a string
    data_str = str(data).encode()

    # Create a hash variable and use it to generate a hash of the transaction data
    hashing = hashlib.sha256()
    hashing.update(data_str)
    data_hash = hashing.hexdigest()

    return data_hash

In [36]:
df_Hash['reportId'] = df_Hash['data'].apply(hash_data)

<h5>
    <b> Map Column Names </b>
</h5> 

In [37]:
opener                  = df['opener'].astype(str).tolist()
rri                     = df['xmlns:rri'].astype(str).tolist()
xsi                     = df['xmlns:xsi'].astype(str).tolist()
report_id               = df_Hash['reportId'].astype(str).tolist()
Operator_Id             = df['OperatorId'].astype(str).tolist()
period_Start            = df['summary_date'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
version                 = df['version'].astype(str).tolist()
partion_id              = df['partionid'].astype(str).tolist()
seq_Number              = df['SeqNumber'].astype(str).tolist()
period_Type             = df['periodType'].astype(str).tolist()
date_Time               = df['dateTime'].astype(str).tolist()
report_Type             = df['reportType'].astype(str).tolist()
brand_Id                = df['brandID'].astype(str).tolist()

# Player
player_id               = df['profile_id'].astype(str).tolist()
Activity_Date_Time      = df['summary_date'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()



# playerActivityDetail
player_Activity_Desc    = df['payment_method'].astype(str).tolist()
player_Activity_Desc_WD = df['ActivityDescWD'].astype(str).tolist()
currency_Code           = df['currencyCode'].astype(str).tolist()

# gameMovement
trans_Class             = df['transClass'].astype(str).tolist()
funds_Type              = df['fundsType'].astype(str).tolist()

trans_Type_dep          = df['ActivityDescDP'].astype(str).tolist()
trans_Type_with         = df['ActivityDescWD'].astype(str).tolist()


trans_Amt_obl          = df['openingBalance'].astype(str).tolist()
trans_Amt_cbl          = df['closingBalance'].astype(str).tolist()
trans_Amt_dep          = df['withdrawal_amt'].astype(str).tolist()
trans_Amt_wth          = df['withdrawal_amt'].astype(str).tolist()

<h5>
    <b> Function To Make XML Readable </b>
</h5> 

In [38]:
def prettify(element, indent='  '):
    queue = [(0, element)]
    while queue:
        level, element = queue.pop(0)
        children = [(level + 1, child) for child in list(element)]
        if children:
            element.text = '\n' + indent * (level+1)
        if queue: 
            element.tail = '\n' + indent * queue[0][0]
        else:
            element.tail = '\n' + indent * (level-1) 
        queue[0:0] = children

<h5>
    <b> Create XML File </b>
</h5> 

In [41]:
xml_doc = ET.Element(name)
gaming_Report = ET.SubElement(xml_doc, 'rri:gamingReport'
                            ,xmlnsrri=rri[0]
                            ,xmlnsxsi=xsi[0]
                            ,version=version[0]
                            ,operatiorId=Operator_Id[0]
                            ,partionId=partion_id[0]
                            ,periodType=period_Type[0]
                            ,periodStart=period_Start[0]
                            ,reportType=report_Type[0]
                            ,reportid=report_id[0]
                            ,seqNumber=seq_Number[0]
                            ,dateTime=date_Time[0])
playerActivity = ET.SubElement(gaming_Report, 'playerActivity' 
                              ,brandId=brand_Id[0])
ET.SubElement(playerActivity,'player'
              ,playerId=player_id[0])
ET.SubElement(playerActivity,'playerActivityDetail'
             ,playerActivity=trans_Type_dep[0]
             ,playerActivityDesc=player_Activity_Desc[0]
             ,playerActivityDateTime=Activity_Date_Time[0])
startingBalanceList = ET.SubElement(playerActivity, 'startingBalanceList')
ET.SubElement(playerActivity, 'summaryBalance' 
              ,fundsType=funds_Type[0]
              ,balanceAmt=trans_Amt_obl[0]
              ,currencyCode=currency_Code[0])
playerMovementList = ET.SubElement(playerActivity, 'playerMovementList')
ET.SubElement(playerMovementList,'playerMovement'
             ,transClass=trans_Class[0]
             ,transType=trans_Type_dep[0]
             ,fundsType=funds_Type[0]
             ,transAmt=trans_Amt_dep[0]
             ,currencyCode=currency_Code[0])
endingBalanceList = ET.SubElement(playerActivity, 'endingBalanceList')
ET.SubElement(endingBalanceList,'summaryBalance'
             ,fundsType=funds_Type[0]
             ,balanceAmt=trans_Amt_cbl[0]
             ,currencyCode=currency_Code[0])
playerMovementList = ET.SubElement(playerActivity, 'playerMovementList')
ET.SubElement(playerActivity, 'summaryBalance' 
              ,fundsType=funds_Type[0]
              ,balanceAmt=trans_Amt_obl[0]
              ,currencyCode=currency_Code[0])
ET.SubElement(playerActivity,'playerActivityDetail'
             ,playerActivity=trans_Type_with[0]
             ,playerActivityDesc=player_Activity_Desc[0]
             ,playerActivityDateTime=Activity_Date_Time[0])
             
playerMovementList = ET.SubElement(playerActivity, 'playerMovementList')
ET.SubElement(playerMovementList,'playerMovement'
             ,transClass=trans_Class[0]
             ,transType=trans_Type_with[0]
             ,fundsType=funds_Type[0]
             ,transAmt=trans_Amt_wth[0]
             ,currencyCode=currency_Code[0])
endingBalanceList = ET.SubElement(playerActivity, 'endingBalanceList')
ET.SubElement(endingBalanceList,'summaryBalance'
             ,fundsType=funds_Type[0]
             ,balanceAmt=trans_Amt_cbl[0]
             ,currencyCode=currency_Code[0])

prettify(xml_doc)

<h5>
    <b> Export XML File To Folder </b>
</h5> 

In [42]:
tree = ET.ElementTree(xml_doc)
tree.write(report_path+title+'_'+serial+'_'+date+'.xml',encoding="utf-8")

In [ ]:
player = []

In [ ]:
for i in player:
    df.